# This is the first Final Assignment of the Applied Data Science 

**We will explore and cluster the neighborhoods in Toronto.**

1. First, we will **scrape** the data with neighborhoods in Toronto, including their Postal Code, Borough, and Neighborhoods, from the Wikipedia [Page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M).
2. We will use the data to create a clean **Pandas DataFrame** corresponding Postal Codes with Neighborhoods.
3. Then we use **geocoder** to extract coordinates for each Neighborhoods for the **Foursquare API** query.
4. Cluster the Neighborhoods  with **scikit-learn**, according to the venues located within a reasonable radius.
5. Present the map with clustered neighborhoods using **Folium**.

* <a href="#part1">Part 1</a>
* <a href="#part2">Part 2</a>
* <a href="#part3">Part 3</a>
* <a href="#part4">Part 4</a>
* <a href="#part5">Part 5</a>

In [1]:
import pandas as pd
import numpy as np

import json # handle json files
import requests # send queries with urls
from urllib.request import urlopen
from lxml import html

from sklearn.cluster import KMeans # cluster the data

import folium # build maps

from bs4 import BeautifulSoup # scrape data

import matplotlib.cm as cm # color the final map
import matplotlib.colors as colors

## Part 1: Scrape the Data
<a name="part1"></a>
Using the **requests** and **BeautifulSoup** libraries.

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wiki = requests.get(url) # send GET request to the wiki page

soup = BeautifulSoup(wiki.text, 'html.parser') # convert the text to an html object
soup = soup.find_all('table') # find all tables in the page

table = soup[0] # take the first one, as it is the one with the data
print(type(table))

<class 'bs4.element.Tag'>


## Part 2: Create a Clean Pandas DataFrame
<a name="part2"></a>
Using the **pandas** and **BeautifulSoup** libraries.

In [40]:
# Iterate over the wiki table and populate corresponding lists
postals = [] # list for Postal Codes
boroughs = [] # list for Boroughs
neighborhoods = [] # list for Neighborhoods
rows = table.find_all('tr') # find elements with the <tr> tag

for row in rows[1:]: # iterate over all those elements (except for the first one as it is empty)
    cells = row.find_all('td') # fine all elements with the <td> tag
    
    # check if Borough is not assigned or Neighborhood is not assigned (see the wiki table)
    if cells[1].text.strip() == 'Not assigned' or cells[2].text.strip() == 'Not assigned':
        continue
        
    postals.append(cells[0].text.strip()) # add all information to corresponding lists
    boroughs.append(cells[1].text.strip())
    
    # there can be several neighborhoods that belong to the same borough, 
    # we need to make sure those are separate entries
    neighborhoods_list = cells[2].text.strip().split(',') # split the third cell in case it has several values
    neighborhoods.append(neighborhoods_list[0].strip())
    
    # if there are several neighborhoods in one cell, add them as separate entries
    if len(neighborhoods_list) > 1:
        i = len(neighborhoods_list) - 1
        while i != 1:
            postals.append(cells[0].text.strip())
            boroughs.append(cells[1].text.strip())
            neighborhoods.append(neighborhoods_list[i].strip())
            i -= 1
            
neighborhoods[0:10]

['Parkwoods',
 'Victoria Village',
 'Regent Park',
 'Lawrence Manor',
 "Queen's Park",
 'Islington Avenue',
 'Malvern',
 'Don Mills',
 'Parkview Hill',
 'Garden District']

In [4]:
neighborhoods = pd.DataFrame({'Postal Code': postals, # Initialize a DataFrame
                             'Boroughs': boroughs,
                             'Neighborhoods': neighborhoods}) 
neighborhoods.head()

,Postal Code,Boroughs,Neighborhoods
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M6A,North York,Lawrence Manor
4,M7A,Downtown Toronto,Queen's Park


In [5]:
neighborhoods.shape

(153, 3)

## Part 3: Extract Coordinates for Each Neighborhood
<a name="part3"></a>
Using the **geocoder** library. The **Goole Geocoding API** did not prove to work, so we will use the data provided by the course.

In [6]:
import geocoder # import geocoder

# WARNING: broken - does not return coordinates for a very long time
def get_lat_lng(postal_code):
    lat_lng_coords = None  # initialize your variable to None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(str(postal_code)))
      lat_lng_coords = g.latlng

    return lat_lng_coords

In [7]:
lat_lng_coords = pd.read_csv('Geospatial_Coordinates.csv') # read the given csv file
lat_lng_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
# merge two dataframes on their common postal codes
neighborhoods = pd.merge(neighborhoods, lat_lng_coords, on='Postal Code')
neighborhoods.head()

,Postal Code,Boroughs,Neighborhoods,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
3,M6A,North York,Lawrence Manor,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


## Part 4: Cluster the Neighborhoods According to the Venues
<a name="part4"></a>
Using the **scikit-learn** library and **Foursquare API**.
### Exploring the data set

In [9]:
print("There are {} unique boroughs".format(len(neighborhoods['Boroughs'].unique())))
print("and {} unique Postal Codes.".format(len(neighborhoods['Postal Code'].unique())))

There are 10 unique boroughs
and 103 unique Postal Codes.


In [10]:
# find the latitude and longitude of Toronto, ON
from geopy.geocoders import Nominatim
address = "Toronto, Canada"

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of Toronto are {}, {}.".format(latitude, longitude))

The geographical coordinates of Toronto are 43.6534817, -79.3839347.


In [11]:
# let's see all of the neighborhoods on the map
map_toronto = folium.Map(location=[latitude, longitude])

label = 'Center of Toronto, Canada'
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
    [latitude, longitude],
    radius=7,
    popup=label,
    color='red',
    fill=True,
    fill_color='red',
    fill_opacity=0.7,
    ).add_to(map_toronto)

# add markers to the map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'],
                                          neighborhoods['Longitude'],
                                          neighborhoods['Boroughs'],
                                          neighborhoods['Neighborhoods']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False,
        ).add_to(map_toronto)

map_toronto

### We can see that the neighborhoods are scattered all across the city, but there is a **high density of neighborhoods** in the Downtown Toronto. 

#### Acknowledge the Incosistency in API Data Fetching
We can see that **First Canadian Place**, **Toronto Dominion Centre**, and **Commerce Court** are practically within blocks of each other, and share many common venues within the 500m. Sharing many commercial venues would mean that they might be clustered together.

As it relates to other neighborhoods, the neighborhoods that are in the **Suburban Areas** of Toronto are quite apart from each other. Therefore, the might not be clustered together as they might not share same venues within the 500m radius.

This means that the approach used might not be precise. Ideally, we would need to have data on the borders of each neighborhood and cluster them according to the venues within those borders.

### Now let's see what kind of data we will extract from the Foursquare API.
The **credentials** will be deactivated after this notebook is released.

In [12]:
CLIENT_ID = '5GNCFSAC2EKFQ1N0DIOUXNFK502JWGIUIRSXSYYZU5XFEX2Y' # your Foursquare ID
CLIENT_SECRET = '3UQQWIB0SP3GMJROGJERI1HIGZOCI04WX4HIQRJ3UNXZTVTC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5GNCFSAC2EKFQ1N0DIOUXNFK502JWGIUIRSXSYYZU5XFEX2Y
CLIENT_SECRET: 3UQQWIB0SP3GMJROGJERI1HIGZOCI04WX4HIQRJ3UNXZTVTC


### Let's first explore venues in one of the neighborhoods.
We will explore the **Parkwoods Neighborhood**.

In [13]:
neighborhood_name = neighborhoods.loc[0, 'Neighborhoods']
neighborhood_name

'Parkwoods'

In [14]:
neighborhood_lat = neighborhoods.loc[0, 'Latitude']
neighborhood_long = neighborhoods.loc[0, 'Longitude']


print('Latitude and longitude values of {}, Toronto are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_lat, 
                                                               neighborhood_long))

Latitude and longitude values of Parkwoods, Toronto are 43.7532586, -79.3296565.


In [15]:
# building an explore Foursquare API query 
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_lat, neighborhood_long, 500 , LIMIT)

In [16]:
results = requests.get(url)
results = results.json()

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']
venues = pd.json_normalize(venues)

# only take related columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
venues = venues.loc[:,filtered_columns]

# extract names of the categories
venues['venue.categories'] = venues.apply(get_category_type, axis=1)

# take only the last word in the column names
venues.columns = [col.split('.')[-1] for col in venues.columns]

venues

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [19]:
print('{} venues were featured'.format(venues.shape[0]))

2 venues were featured


### Explore the rest of the neighborhoods in Toronto

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhoods'],
                                latitudes=neighborhoods['Latitude'],
                                longitudes=neighborhoods['Longitude'],)

Parkwoods
Victoria Village
Regent Park
Lawrence Manor
Queen's Park
Islington Avenue
Malvern
Don Mills
Parkview Hill
Garden District
Glencairn
West Deane Park
Cloverdale
Islington
Martin Grove
Rouge Hill
Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Thorncliffe Park
Richmond
King
Dufferin
Scarborough Village
Fairview
Oriole
Northwood Park
East Toronto
Harbourfront East
Toronto Islands
Little Portugal
Kennedy Park
East Birchmount Park
Bayview Village
Downsview
The Danforth West
Toronto Dominion Centre
Brockton
Exhibition Place
Golden Mile
Oakridge
York Mills
Downsview
India Bazaar
Commerce Court
North Park
Upwood Park
Humber Summit
Cliffside
Scarborough Village West
Willowdale
Downsview
Studio District
Bedford Park
Del Ray
Keelsdale and Silverthorn
Humber

In [22]:
print(toronto_venues.shape)
toronto_venues.head()

(2849, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [23]:
# how many venues were returned for each neighborhood?
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
Albion Gardens,9,9,9,9,9,9
Alderwood,7,7,7,7,7,7
Bathurst Manor,21,21,21,21,21,21
Bathurst Quay,17,17,17,17,17,17
...,...,...,...,...,...,...
Woburn,4,4,4,4,4,4
Woodbine Heights,6,6,6,6,6,6
York Mills,1,1,1,1,1,1


In [24]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 267 uniques categories.


### Analyze each neighborhood venue categories

In [25]:
pd.set_option('display.max_columns', None)

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix='', prefix_sep='')

# adding neighborhood column to the front
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
index = toronto_onehot.columns.get_loc("Neighborhood")
fixed_columns = [toronto_onehot.columns[index]] + list(toronto_onehot.columns[:index]) + list(toronto_onehot.columns[index+1:])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [26]:
toronto_onehot.shape

(2849, 267)

In [27]:
# Group by Neighborhoods and find the mean between all columns for each row
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,

In [28]:
toronto_grouped

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,

### The five top venues in each neighborhood.

In [29]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1                     Lounge  0.25
2         Chinese Restaurant  0.25
3  Latin American Restaurant  0.25
4          Accessories Store  0.00


----Albion Gardens----
                 venue  freq
0          Pizza Place  0.22
1        Grocery Store  0.22
2             Pharmacy  0.11
3           Beer Store  0.11
4  Fried Chicken Joint  0.11


----Alderwood----
            venue  freq
0     Pizza Place  0.29
1             Gym  0.14
2  Sandwich Place  0.14
3             Pub  0.14
4    Skating Rink  0.14


----Bathurst Manor----
                       venue  freq
0                       Bank  0.10
1                Coffee Shop  0.10
2                Gas Station  0.05
3                      Diner  0.05
4  Middle Eastern Restaurant  0.05


----Bathurst Quay----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3             Plane  0.06
4           Airport 

                           venue  freq
0                 Baseball Field   0.5
1  Paper / Office Supplies Store   0.5
2              Accessories Store   0.0
3      Middle Eastern Restaurant   0.0
4            Monument / Landmark   0.0


----Humewood-Cedarvale----
               venue  freq
0              Field  0.25
1               Park  0.25
2       Hockey Arena  0.25
3              Trail  0.25
4  Accessories Store  0.00


----India Bazaar----
                  venue  freq
0  Fast Food Restaurant  0.10
1            Board Shop  0.05
2        Ice Cream Shop  0.05
3        Sandwich Place  0.05
4            Steakhouse  0.05


----Island airport----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3             Plane  0.06
4           Airport  0.06


----Jamestown----
                 venue  freq
0          Pizza Place  0.22
1        Grocery Store  0.22
2             Pharmacy  0.11
3           Beer Store  0.11
4  Fried Chicken Joint  0.

                       venue  freq
0                       Park  0.33
1                       Pool  0.33
2                      River  0.33
3          Accessories Store  0.00
4  Middle Eastern Restaurant  0.00


----The Queensway East----
                 venue  freq
0        Deli / Bodega   0.5
1       Baseball Field   0.5
2    Accessories Store   0.0
3   Miscellaneous Shop   0.0
4  Monument / Landmark   0.0


----Thistletown----
                 venue  freq
0          Pizza Place  0.22
1        Grocery Store  0.22
2             Pharmacy  0.11
3           Beer Store  0.11
4  Fried Chicken Joint  0.11


----Thorncliffe Park----
               venue  freq
0  Indian Restaurant  0.10
1     Sandwich Place  0.10
2        Yoga Studio  0.05
3        Pizza Place  0.05
4               Park  0.05


----Toronto Dominion Centre----
         venue  freq
0  Coffee Shop  0.10
1        Hotel  0.08
2         Café  0.06
3  Salad Place  0.04
4   Restaurant  0.03


----Toronto Islands----
                

### The top ten venues in each neighborhood.

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Chinese Restaurant,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Electronics Store
1,Albion Gardens,Grocery Store,Pizza Place,Pharmacy,Beer Store,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop
2,Alderwood,Pizza Place,Gym,Sandwich Place,Coffee Shop,Skating Rink,Pub,Dog Run,Dim Sum Restaurant,Diner,Discount Store
3,Bathurst Manor,Coffee Shop,Bank,Frozen Yogurt Shop,Shopping Mall,Bridal Shop,Sandwich Place,Diner,Restaurant,Middle Eastern Restaurant,Supermarket
4,Bathurst Quay,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boat or Ferry,Coffee Shop,Plane,Sculpture Garden,Rental Car Location,Boutique


### Time to cluster.

In [32]:
# set the number of clusters (groups)
kclusters = 5

# create new dataframe without neighborhood names
toronto_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering algorithms
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

kmeans.labels_[:10]

array([2, 0, 2, 2, 2, 2, 0, 2, 2, 2], dtype=int32)

In [33]:
# add cluster labels to the DataFrame
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# rename the initial dataframe to have a neighborhood column for merge
neighborhoods.rename(columns={'Neighborhoods': 'Neighborhood'}, inplace=True)

# merge two DataFrames on common Neighborhood column
toronto_merged = pd.merge(neighborhoods, neighborhoods_venues_sorted, on='Neighborhood')
toronto_merged.head()

,Postal Code,Boroughs,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4,Food & Drink Shop,Park,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
1,M4A,North York,Victoria Village,43.725882,-79.315572,2,Portuguese Restaurant,French Restaurant,Coffee Shop,Hockey Arena,Intersection,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,2,Coffee Shop,Park,Bakery,Café,Breakfast Spot,Pub,Theater,Yoga Studio,Brewery,Shoe Store
3,M6A,North York,Lawrence Manor,43.718518,-79.464763,2,Clothing Store,Accessories Store,Boutique,Furniture / Home Store,Event Space,Coffee Shop,Women's Store,Vietnamese Restaurant,Filipino Restaurant,Field
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,2,Coffee Shop,Sushi Restaurant,Yoga Studio,Creperie,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Portuguese Restaurant


## Part 5: Present the Map with Clustered Labels
<a name="part5"></a>

In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Boroughs,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,0,Fast Food Restaurant,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore
99,Scarborough,0,Pizza Place,Fast Food Restaurant,Fried Chicken Joint,Gas Station,Intersection,Italian Restaurant,Rental Car Location,Thai Restaurant,Chinese Restaurant,Bank
100,Scarborough,0,Pizza Place,Fast Food Restaurant,Fried Chicken Joint,Gas Station,Intersection,Italian Restaurant,Rental Car Location,Thai Restaurant,Chinese Restaurant,Bank
119,Etobicoke,0,Grocery Store,Pizza Place,Pharmacy,Beer Store,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop
120,Etobicoke,0,Grocery Store,Pizza Place,Pharmacy,Beer Store,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop
121,Etobicoke,0,Grocery Store,Pizza Place,Pharmacy,Beer Store,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop
122,Etobicoke,0,Grocery Store,Pizza Place,Pharmacy,Beer Store,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop
123,Etobicoke,0,Grocery Store,Pizza Place,Pharmacy,Beer Store,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop
124,Etobicoke,0,Grocery Store,Pizza Place,Pharmacy,Beer Store,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop
125,Etobicoke,0,Grocery Store,Pizza Place,Pharmacy,Beer Store,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop


In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Boroughs,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
73,North York,1,Paper / Office Supplies Store,Baseball Field,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
136,Etobicoke,1,Deli / Bodega,Baseball Field,Yoga Studio,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Diner
137,Etobicoke,1,Deli / Bodega,Baseball Field,Yoga Studio,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Diner
138,Etobicoke,1,Deli / Bodega,Baseball Field,Yoga Studio,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Diner
139,Etobicoke,1,Deli / Bodega,Baseball Field,Yoga Studio,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Diner
140,Etobicoke,1,Deli / Bodega,Baseball Field,Yoga Studio,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Diner
141,Etobicoke,1,Deli / Bodega,Baseball Field,Yoga Studio,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Diner
142,Etobicoke,1,Deli / Bodega,Baseball Field,Yoga Studio,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Diner


In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Boroughs,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,2,Portuguese Restaurant,French Restaurant,Coffee Shop,Hockey Arena,Intersection,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
2,Downtown Toronto,2,Coffee Shop,Park,Bakery,Café,Breakfast Spot,Pub,Theater,Yoga Studio,Brewery,Shoe Store
3,North York,2,Clothing Store,Accessories Store,Boutique,Furniture / Home Store,Event Space,Coffee Shop,Women's Store,Vietnamese Restaurant,Filipino Restaurant,Field
4,Downtown Toronto,2,Coffee Shop,Sushi Restaurant,Yoga Studio,Creperie,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Portuguese Restaurant
6,North York,2,Gym,Beer Store,Japanese Restaurant,Restaurant,Coffee Shop,Caribbean Restaurant,Athletics & Sports,Supermarket,Sporting Goods Shop,Sushi Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
135,East Toronto,2,Light Rail Station,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Pizza Place
143,Etobicoke,2,Grocery Store,Discount Store,Fast Food Restaurant,Burger Joint,Sandwich Place,Burrito Place,Supplement Shop,Bakery,Tanning Salon,Wings Joint
144,Etobicoke,2,Grocery Store,Discount Store,Fast Food Restaurant,Burger Joint,Sandwich Place,Burrito Place,Supplement Shop,Bakery,Tanning Salon,Wings Joint
145,Etobicoke,2,Grocery Store,Discount Store,Fast Food Restaurant,Burger Joint,Sandwich Place,Burrito Place,Supplement Shop,Bakery,Tanning Salon,Wings Joint


In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Boroughs,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
79,York,3,Jewelry Store,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore


In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Boroughs,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,4,Food & Drink Shop,Park,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
16,York,4,Field,Park,Hockey Arena,Trail,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
24,York,4,Park,Women's Store,Pool,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Eastern European Restaurant
41,East York,4,Park,Coffee Shop,Convenience Store,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
61,North York,4,Park,Bakery,Construction & Landscaping,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
62,North York,4,Park,Bakery,Construction & Landscaping,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
75,Central Toronto,4,Park,Bus Line,Swim School,Dim Sum Restaurant,Yoga Studio,Doner Restaurant,Discount Store,Distribution Center,Dog Run,Drugstore
82,North York,4,Park,Convenience Store,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Electronics Store
84,Central Toronto,4,Park,Trail,Bus Line,Sushi Restaurant,Jewelry Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dim Sum Restaurant
93,Etobicoke,4,Park,Sandwich Place,Mobile Phone Shop,Bus Line,Yoga Studio,Doner Restaurant,Discount Store,Distribution Center,Dog Run,Donut Shop
